## Azure AI Agents with Native and Prompt Template Plugins

![image](./Assets/prompt-and-native-plugins.png)

### SDK and toolset installation

In [ ]:
%pip install semantic-kernel==1.28.1

### Referencing the Important Libraries and Packages

In [ ]:
from semantic_kernel.agents import ChatCompletionAgent
from semantic_kernel.connectors.ai import FunctionChoiceBehavior
from semantic_kernel.connectors.ai.open_ai import AzureChatCompletion, AzureChatPromptExecutionSettings
from semantic_kernel.functions import KernelFunctionFromPrompt
from semantic_kernel.kernel import Kernel
from dotenv import load_dotenv
import os
import math
from typing import Annotated

from semantic_kernel.functions.kernel_function_decorator import kernel_function

### Creating the Math Native Plugin

In [ ]:
class Math:
    """
    Description: MathPlugin provides a set of functions to make Math calculations.

    Usage:
        kernel.add_plugin(MathPlugin(), plugin_name="math")

    Examples:
        {{math.Add}} => Returns the sum of input and amount (provided in the KernelArguments)
        {{math.Subtract}} => Returns the difference of input and amount (provided in the KernelArguments)
        {{math.Multiply}} => Returns the multiplication of input and number2 (provided in the KernelArguments)
        {{math.Divide}} => Returns the division of input and number2 (provided in the KernelArguments)
    """

    @kernel_function(
        description="Divide two numbers.",
        name="Divide",
    )
    def divide(
        self,
        number1: Annotated[float, "the first number to divide from"],
        number2: Annotated[float, "the second number to by"],
    ) -> Annotated[float, "The output is a float"]:
        return float(number1) / float(number2)

    @kernel_function(
        description="Multiply two numbers. When increasing by a percentage, don't forget to add 1 to the percentage.",
        name="Multiply",
    )
    def multiply(
        self,
        number1: Annotated[float, "the first number to multiply"],
        number2: Annotated[float, "the second number to multiply"],
    ) -> Annotated[float, "The output is a float"]:
        return float(number1) * float(number2)

    @kernel_function(
        description="Takes the square root of a number",
        name="Sqrt",
    )
    def square_root(
        self,
        number1: Annotated[float, "the number to take the square root of"],
    ) -> Annotated[float, "The output is a float"]:
        return math.sqrt(float(number1))

    @kernel_function(name="Add")
    def add(
        self,
        number1: Annotated[float, "the first number to add"],
        number2: Annotated[float, "the second number to add"],
    ) -> Annotated[float, "the output is a float"]:
        return float(number1) + float(number2)

    @kernel_function(
        description="Subtracts value to a value",
        name="Subtract",
    )
    def subtract(
        self,
        number1: Annotated[float, "the first number"],
        number2: Annotated[float, "the number to subtract"],
    ) -> Annotated[float, "the output is a float"]:
        return float(number1) - float(number2)

### Creating the Kernel with Auto Function Behavior and the native and prompt-template plugins

In [ ]:
from semantic_kernel.agents import AzureAIAgent, AzureAIAgentSettings
from azure.ai.projects import AIProjectClient
from azure.ai.projects.models import MessageTextContent
from azure.identity import DefaultAzureCredential

load_dotenv()

model = os.getenv("AZURE_OPENAI_CHAT_COMPLETION_MODEL")

project_client = AzureAIAgent.create_client(credential=DefaultAzureCredential(),
                           conn_str=os.getenv("PROJECT_CONNECTION_STRING")
)

kernel = Kernel()

service_id = "default"

kernel.add_service(
    AzureChatCompletion(service_id=service_id,
                        api_key="",
                        deployment_name="",
                        endpoint = ""
    )
)

# Get the AI Service settings
settings = kernel.get_prompt_execution_settings_from_service_id(service_id=service_id)

# Configure the function choice behavior to auto invoke kernel functions
settings.function_choice_behavior = FunctionChoiceBehavior.Auto()

#Adding the native plugin to the kernel
kernel.add_plugin(Math(), plugin_name="math")

#Adding the prompt template plugin to the kernel
plugin = kernel.add_plugin(parent_directory="../plugins/prompt_templates/", plugin_name="basic_plugin")

### Creating a ChatCompletionAgent with the kernel

In [ ]:
from azure.identity import DefaultAzureCredential
from azure.ai.projects import AIProjectClient
from semantic_kernel.functions import KernelArguments

# Create the agent
agent = ChatCompletionAgent(
    kernel=kernel, 
    name="Plugins-Agent", 
    instructions="You are a helpful AI assistant", 
    arguments=KernelArguments(settings=settings),
)

### Invoking the agent to see plugin execution

In [ ]:
user_input = """add 5 and 2. Also create contact information for the user with the following details:
1) Name: John Doe
2) Phone Number: 123-456-7890
3) Email ID: john@outlook.com
4) Address: 123 Main St, Springfield, USA
"""
# 3. Call the agent with user input
async def get_response_from_agent():
    response =  await agent.get_response(
        messages = user_input
    )
    
    return response

response = await get_response_from_agent()
print(response)

### Viewing Intermediate plugin execution steps with "intermediate_steps"

In [ ]:
from semantic_kernel.contents import ChatMessageContent
from semantic_kernel.agents import ChatHistoryAgentThread
from semantic_kernel.contents import AuthorRole, ChatMessageContent, FunctionCallContent, FunctionResultContent

# Define the thread
thread = ChatHistoryAgentThread()

# Define a list to hold callback message content
intermediate_steps: list[ChatMessageContent] = []

# Define an async method to handle the `on_intermediate_message` callback
async def handle_intermediate_steps(message: ChatMessageContent) -> None:
    intermediate_steps.append(message)

# Define the main function to get a response
async def get_response_from_agent_with_intermediate_steps():
    async for response in agent.invoke(
        messages=user_input,
        thread=thread,
        on_intermediate_message=handle_intermediate_steps
    ):
        return response

# Run the function
response = await get_response_from_agent_with_intermediate_steps()
print("Agent response: \n")
print(f"{response} \n")
print("------------------------------------------------------------")

 # Print the intermediate steps
print("\nIntermediate Steps:")
for msg in intermediate_steps:
      if any(isinstance(item, FunctionResultContent) for item in msg.items):
          for fr in msg.items:
              if isinstance(fr, FunctionResultContent):
                  print(f"Function Result:> {fr.result} for function: {fr.name}")
      elif any(isinstance(item, FunctionCallContent) for item in msg.items):
          for fcc in msg.items:
              if isinstance(fcc, FunctionCallContent):
                  print(f"Function Call:> {fcc.name} with arguments: {fcc.arguments}")
      else:
          print(f"{msg.role}: {msg.content}")